In [1]:
# import
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.cm as cm 
import seaborn as sns
import pandas as pd
from scipy import signal
from importlib import reload

#import winsound as sd
import pickle
import sys
import os

In [2]:
# load
cd = os.getcwd()
path = os.path.dirname(cd)

In [3]:
def calculate_success(CA3_R_list, CA1_R_list, CA3, Data, CN):
    Trial = 50
    Trial_CA3 = np.zeros(Trial)
    Trial_CA1 = np.zeros(Trial)
    Inter_CA1 = np.zeros(Trial)
    # CA3 firing rate
    N = 3
    # CA3 firing ratio
    R = 0.3
    Success = 0
    S_list = []
    Weird = 0
    Interference = 0
    Fail = 0
    Classify_num = CN
    Winner = np.zeros(Classify_num)

    # 각 trial 에서 CA3의 firing rate N 보다 큰 애들 인덱스
    # 각 엔그램이랑 겹치는 정도 R
    for i, comp in enumerate(CA3_R_list):
        temp = np.where(comp >= N)[0]
        # firing neuron과 target engram과의 overlap 정도 비교
        # 어느 하나로 수렴했으면 +1
        # 여러 엔그램으로 수렴했으면 1< 일 것
        for j, ca3 in enumerate(CA3):
            if len(set(temp).intersection(set(ca3)))/len(ca3) >= R:
                Trial_CA3[i] += 1

    # 각 trial 에서 CA1의 firing rate 1보다 큰 애들 인덱스
    # 특정 인풋이랑 완전히 일치해야만 성공
    for i, comp in enumerate(CA1_R_list):
        # 일단 발화했으면 다 체크
        temp = np.where(comp >= 1)[0]
        # 그리고 최종적으로 발화한 애들이 2보단 많이 발화해야함
        temp2 = np.where(comp >= 2)[0]
        
        for j, data in enumerate(Data):
            # 일단 발화한 애들이 어떤 data로 수렴했고
            if len(temp) == len(data) and len(set(temp).intersection(set(data))) == len(temp):
                # 얘네가 충분히 rate가 크다면 +1
                if len(temp) == len(temp2):
                    Trial_CA1[i] += 1
                    Winner[j] += 1
         # interference 체크. CA1이 발화를 아예 안했을 경우랑 구분 짓기 위해.
        if len(set(temp2)) !=0:
            Inter_CA1[i] += 1
        
    for n in range(Trial):
        if Trial_CA1[n] == 1:
            Success += 1
            S_list.append(1)
        # CA3에서 여러 engram이 발화하면서 성공도 못했을 때
        # 여러 engram이 발화했어도 CA1에 영향없으면 괜찮기 때문
        elif Inter_CA1[n] != 0 and Trial_CA3[n] > 1:
            Interference += 1
            S_list.append(0)
        else:
            Fail += 1
            S_list.append(0)
    print('Success : ', Success)
    #print('Weird : ', Weird)
    print('Interference : ', Interference)
    print('Fail : ', Fail)
    return Success, Weird, Interference, Fail, Winner

In [5]:
# load
cd = os.getcwd()
path = os.path.dirname(cd)

with open(path+'//Data_final.pkl', 'rb') as file:
    Filt_idata = pickle.load(file)
    Filt_gdata = pickle.load(file)
    Filt_glen = pickle.load(file)
    F_DataLen = pickle.load(file)
    
    Af_py = pickle.load(file)
    Af_in = pickle.load(file)
    
    In_py_list = pickle.load(file)
    In_in_list = pickle.load(file)

In [ ]:
# n : 학습된 정보 종류
# m : bias 정도
path_results = '//module//Results_various_cond4'
for n in range(5):
    for m in range(5):
        with open(path+path_results+'//Results-biasG_v4%d-%d.pkl' % (n+1,m), 'rb') as file:
            Success = pickle.load(file)
            Fail = pickle.load(file)
            If_fail = pickle.load(file)
            Winfin = pickle.load(file)
            
            globals()["CA3_R_list_bG{}_{}".format(n+1,m)] = pickle.load(file)
            globals()["CA1_R_list_bG{}_{}".format(n+1,m)] = pickle.load(file)

In [ ]:
for n in range(5):
    for m in range(5):
        with open(path+path_results+'//Results-biasM_v4%d-%d.pkl' % (n+1,m), 'rb') as file:
            Success = pickle.load(file)
            Fail = pickle.load(file)
            If_fail = pickle.load(file)
            Winfin = pickle.load(file)
            
            globals()["CA3_R_list_bM{}_{}".format(n+1,m)] = pickle.load(file)
            globals()["CA1_R_list_bM{}_{}".format(n+1,m)] = pickle.load(file)

In [ ]:
Repeat = 50

In [ ]:
# Bias 조사
with open('Cue_and_Target_Bias_v4.pkl', 'rb') as file:
    Target_Data = pickle.load(file)
    Cue_Data = pickle.load(file)

In [ ]:
Target_temp = [Target_Data[0], Target_Data[5], Target_Data[10], Target_Data[15], Target_Data[20]]

for n in range(5):        
    globals()["Data_bias{}".format(n)] = []
    globals()["CA3_bias{}".format(n)] = []
    for j, comp in enumerate(Target_temp[n]):
        globals()["Data_bias{}".format(n)].append(np.where(Filt_idata[:,comp] !=0)[0])
        globals()["CA3_bias{}".format(n)].append(Af_py[comp])

In [ ]:
N = 2
for n in range(5):
    for m in range(5):
        globals()["Success_bG{}_{}".format(n+1,m)], globals()["W_bG{}_{}".format(n+1,m)], globals()["If_bG{}_{}".format(n+1,m)], globals()["F_bG{}_{}".format(n+1,m)], globals()["Win_bG{}_{}".format(n+1,m)] =  calculate_success(
        globals()["CA3_R_list_bG{}_{}".format(n+1,m)], 
        globals()["CA1_R_list_bG{}_{}".format(n+1,m)], 
        globals()["CA3_bias{}".format(n)], 
        globals()["Data_bias{}".format(n)], N)
Success_bG = []
Success_bG_list = []
Success_bG_idx = []
for m in range(5):
    Success_temp = []
    for n in range(5):
        Success_temp.append(globals()["Success_bG{}_{}".format(n+1,m)]*100/Repeat)
        Success_bG_idx.append([n+1,m])
    Success_bG.append(np.mean(Success_temp))
    Success_bG_list += Success_temp

In [ ]:
N = 2
for m in range(5):
    for n in range(5):
        globals()["Success_bM{}_{}".format(n+1,m)], globals()["W_bM{}_{}".format(n+1,m)], globals()["If_bM{}_{}".format(n+1,m)], globals()["F_bM{}_{}".format(n+1,m)], globals()["Win_bM{}_{}".format(n+1,m)] =  calculate_success(
        globals()["CA3_R_list_bM{}_{}".format(n+1,m)], 
        globals()["CA1_R_list_bM{}_{}".format(n+1,m)], 
        globals()["CA3_bias{}".format(n)], 
        globals()["Data_bias{}".format(n)], N)
        
Success_bM = []
Success_bM_list = []
for m in range(5):
    Success_temp = []
    for n in range(5):
        Success_temp.append(globals()["Success_bM{}_{}".format(n+1,m)]*100/Repeat)
    Success_bM.append(np.mean(Success_temp))
    Success_bM_list += Success_temp

In [ ]:
Success_bM_F = []
for n in range(5):
    temp = Success_bM_list[5*n:5*(n+1)]
    Success_bM_F.append(np.mean(temp))
Success_bG_F = []
for n in range(5):
    temp = Success_bG_list[5*n:5*(n+1)]
    Success_bG_F.append(np.mean(temp))    

In [ ]:
data_dic = {}
data_dic['Input bias level'] = [0,1,2,3,4,0,1,2,3,4]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(5):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(5):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')

data_dic['Performance (%)'] = np.array(Success_bM_F+Success_bG_F)

df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.lineplot(data = df, x='Input bias level', y='Performance (%)', marker='o', markersize=10, palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.savefig('Fig-Input biasf.png', bbox_inches='tight')
plt.show()

In [ ]:
data_dic = {}
data_dic['Input bias level'] = []
for n in range(10):
    m = n%5
    data_dic['Input bias level'] += [m,m,m,m,m]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(25):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(25):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')

data_dic['Performance (%)'] = Success_bM_list+Success_bG_list

df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.lineplot(data = df, x='Input bias level', y='Performance (%)', marker='o', markersize=10, palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.show()

In [ ]:
data_dic = {}
data_dic['Input bias level'] = []
for n in range(10):
    m = n%5
    data_dic['Input bias level'] += [m,m,m,m,m]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(25):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(25):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')

data_dic['Performance (%)'] = Success_bM_list+Success_bG_list

df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.scatterplot(data = df, x='Input bias level', y='Performance (%)', marker='o', palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.show()

In [ ]:
def cal_overlap(total,A,B):
    if (len(A)+len(B)) !=0:
        output = len(A-(total- B))*2 / (len(A)+len(B))
    else:
        output = 0
    return output

In [ ]:
with open('Cue_and_Target_Overlap_v4.pkl', 'rb') as file:
    Target_Data = pickle.load(file)
    Cue_Data = pickle.load(file)

In [ ]:
for n in range(6):
    for m in range(5):
        with open(path+path_results+'//Results-overlapG_v4%d-%d.pkl' % (n+1,m), 'rb') as file:
            Success = pickle.load(file)
            Fail = pickle.load(file)
            If_fail = pickle.load(file)
            Winfin = pickle.load(file)
            
            globals()["CA3_R_list_oG{}_{}".format(n+1,m)] = pickle.load(file)
            globals()["CA1_R_list_oG{}_{}".format(n+1,m)] = pickle.load(file)

In [ ]:
for n in range(6):
    for m in range(5):
        with open(path+path_results+'//Results-overlapM_v4%d-%d.pkl' % (n+1,m), 'rb') as file:
            Success = pickle.load(file)
            Fail = pickle.load(file)
            If_fail = pickle.load(file)
            Winfin = pickle.load(file)
            
            globals()["CA3_R_list_oM{}_{}".format(n+1,m)] = pickle.load(file)
            globals()["CA1_R_list_oM{}_{}".format(n+1,m)] = pickle.load(file)

In [ ]:
# overlap 0
Target_temp0 = Target_Data[:5]
# overlap 1 / 0.5 - 1
Target_temp1 = Target_Data[5:10]
# overlap 2 / 1 - 1.5
Target_temp2 = Target_Data[10:15]
# overlap 3 / 1.5 - 2
Target_temp3 = Target_Data[15:20]
# overlap 4 / 2 - 2.5
Target_temp4 = Target_Data[20:25]
# overlap 5 / 2.5 - 3
Target_temp5 = Target_Data[25:30]

In [ ]:
CA3_overlap_mean = []
for n in range(6):
    overlap_temp = []
    for j, comp in enumerate(globals()["Target_temp{}".format(n)]):
        overlap_temp.append(cal_overlap(set(range(2400)), set(Af_py[comp[0]]), set(Af_py[comp[1]])))
    CA3_overlap_mean.append(np.mean(overlap_temp))

In [ ]:
for n in range(6):
    for m in range(5):
        globals()["Data_over{}_{}".format(n,m)] = []
        globals()["CA3_over{}_{}".format(n,m)] = []
        for j, comp in enumerate(globals()["Target_temp{}".format(n)][m]):
            globals()["Data_over{}_{}".format(n,m)].append(np.where(Filt_idata[:,comp] !=0)[0])
            globals()["CA3_over{}_{}".format(n,m)].append(Af_py[comp])

In [ ]:
N = 2
for n in range(6):
    for m in range(5):
        globals()["Success_oG{}_{}".format(n+1,m)], globals()["W_oG{}_{}".format(n+1,m)], globals()["If_oG{}_{}".format(n+1,m)], globals()["F_oG{}_{}".format(n+1,m)], globals()["W_oG{}_{}".format(n+1,m)] =  calculate_success(
        globals()["CA3_R_list_oG{}_{}".format(n+1,m)], 
        globals()["CA1_R_list_oG{}_{}".format(n+1,m)], 
        globals()["CA3_over{}_{}".format(n,m)], 
        globals()["Data_over{}_{}".format(n,m)], N)
        
Success_oG = []
Success_oG_list = []
for n in range(6):
    Success_temp = []
    for m in range(5):
        Success_temp.append(globals()["Success_oG{}_{}".format(n+1,m)]*100/Repeat)
    Success_oG.append(np.mean(Success_temp))
    Success_oG_list += Success_temp

In [ ]:
N = 2
for n in range(6):
    for m in range(5):
        globals()["Success_oM{}_{}".format(n+1,m)], globals()["W_oM{}_{}".format(n+1,m)], globals()["If_oM{}_{}".format(n+1,m)], globals()["F_oM{}_{}".format(n+1,m)], globals()["W_oM{}_{}".format(n+1,m)] =  calculate_success(
        globals()["CA3_R_list_oM{}_{}".format(n+1,m)], 
        globals()["CA1_R_list_oM{}_{}".format(n+1,m)], 
        globals()["CA3_over{}_{}".format(n,m)], 
        globals()["Data_over{}_{}".format(n,m)], N)
        
Success_oM = []
Success_oM_list = []
for n in range(6):
    Success_temp = []
    for m in range(5):
        Success_temp.append(globals()["Success_oM{}_{}".format(n+1,m)]*100/Repeat)
    Success_oM.append(np.mean(Success_temp))
    Success_oM_list += Success_temp

In [ ]:
Success_oM_F = []
for n in range(6):
    temp = Success_oM_list[5*n:5*(n+1)]
    Success_oM_F.append(np.mean(temp))
Success_oG_F = []
for n in range(6):
    temp = Success_oG_list[5*n:5*(n+1)]
    Success_oG_F.append(np.mean(temp))   

In [ ]:
data_dic = {}
data_dic['Overlap ratio of CA3 engram (%)'] = CA3_overlap_mean+CA3_overlap_mean
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(6):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(6):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')

data_dic['Performance (%)'] = np.array(Success_oM_F+Success_oG_F)

df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.lineplot(data = df, x='Overlap ratio of CA3 engram (%)', y='Performance (%)', marker='o', markersize=10, palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.savefig('Fig-Overlapf.png', bbox_inches='tight')
plt.show()

In [ ]:
data_dic = {}
data_dic['Overlap ratio of CA3 engram (%)'] = []
for n in range(12):
    m = n%6
    data_dic['Overlap ratio of CA3 engram (%)'] += [CA3_overlap_mean[m],CA3_overlap_mean[m],CA3_overlap_mean[m],CA3_overlap_mean[m],CA3_overlap_mean[m]]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(30):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(30):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')

data_dic['Performance (%)'] = Success_oM_list+Success_oG_list

df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.lineplot(data = df, x='Overlap ratio of CA3 engram (%)', y='Performance (%)', marker='o', markersize=10, palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.show()

In [ ]:
data_dic = {}
data_dic['Overlap ratio of CA3 engram (%)'] = []
for n in range(12):
    m = n%6
    data_dic['Overlap ratio of CA3 engram (%)'] += [CA3_overlap_mean[m],CA3_overlap_mean[m],CA3_overlap_mean[m],CA3_overlap_mean[m],CA3_overlap_mean[m]]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(30):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(30):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')

data_dic['Performance (%)'] = Success_oM_list+Success_oG_list

df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.scatterplot(data = df, x='Overlap ratio of CA3 engram (%)', y='Performance (%)', marker='o', palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.show()

In [ ]:
for n in range(5):
    for m in range(5):
        with open(path+path_results+'//Results-sizeG_v4%d-%d.pkl' % (n+1,m), 'rb') as file:
            Success = pickle.load(file)
            Fail = pickle.load(file)
            If_fail = pickle.load(file)
            
            globals()["Winfin_sG{}_{}".format(n+1,m)] = pickle.load(file)
            
            globals()["CA3_R_list_sG{}_{}".format(n+1,m)] = pickle.load(file)
            globals()["CA1_R_list_sG{}_{}".format(n+1,m)] = pickle.load(file)

In [ ]:
for n in range(5):
    for m in range(5):
        with open(path+path_results+'//Results-sizeM_v4%d-%d.pkl' % (n+1,m), 'rb') as file:
            Success = pickle.load(file)
            Fail = pickle.load(file)
            If_fail = pickle.load(file)
            
            globals()["Winfin_sM{}_{}".format(n+1,m)] = pickle.load(file)
            
            globals()["CA3_R_list_sM{}_{}".format(n+1,m)] = pickle.load(file)
            globals()["CA1_R_list_sM{}_{}".format(n+1,m)] = pickle.load(file)

In [ ]:
# Bias 조사
with open('Cue_and_Target_Size_v4.pkl', 'rb') as file:
    Target_Data = pickle.load(file)
    Cue_Data = pickle.load(file)

In [ ]:
# size difference 1
Target_temp0 = Target_Data[:5]
# size difference 2 
Target_temp1 = Target_Data[5:10]
# size difference 3 
Target_temp2 = Target_Data[10:15]
# size difference 4 
Target_temp3 = Target_Data[15:20]
# size difference 5 
Target_temp4 = Target_Data[20:25]

In [ ]:
# Target data, engram 정리
for n in range(5):
    for m in range(5):
        globals()["Data_size{}_{}".format(n,m)] = []
        globals()["CA3_size{}_{}".format(n,m)] = []
        for j, comp in enumerate(globals()["Target_temp{}".format(n)][m]):
            globals()["Data_size{}_{}".format(n,m)].append(np.where(Filt_idata[:,comp] !=0)[0])
            globals()["CA3_size{}_{}".format(n,m)].append(Af_py[comp])

In [ ]:
N = 2
for n in range(5):
    for m in range(5):
        globals()["Success_sG{}_{}".format(n+1,m)], globals()["W_sG{}_{}".format(n+1,m)], globals()["If_sG{}_{}".format(n+1,m)], globals()["F_sG{}_{}".format(n+1,m)], globals()["Win_sG{}_{}".format(n+1,m)] =  calculate_success(
        globals()["CA3_R_list_sG{}_{}".format(n+1,m)], 
        globals()["CA1_R_list_sG{}_{}".format(n+1,m)], 
        globals()["CA3_size{}_{}".format(n,m)], 
        globals()["Data_size{}_{}".format(n,m)], N)
        
Success_sG = []
Success_sG_list = []
for n in range(5):
    Success_temp = []
    for m in range(5):
        Success_temp.append(globals()["Success_sG{}_{}".format(n+1,m)]*100/Repeat)
    Success_sG.append(np.mean(Success_temp))
    Success_sG_list += Success_temp

In [ ]:
N = 2
for n in range(5):
    for m in range(5):
        globals()["Success_sM{}_{}".format(n+1,m)], globals()["W_sM{}_{}".format(n+1,m)], globals()["If_sM{}_{}".format(n+1,m)], globals()["F_sM{}_{}".format(n+1,m)], globals()["Win_sM{}_{}".format(n+1,m)] =  calculate_success(
        globals()["CA3_R_list_sM{}_{}".format(n+1,m)], 
        globals()["CA1_R_list_sM{}_{}".format(n+1,m)], 
        globals()["CA3_size{}_{}".format(n,m)], 
        globals()["Data_size{}_{}".format(n,m)], N)
        
Success_sM = []
Success_sM_list = []
for n in range(5):
    Success_temp = []
    for m in range(5):
        Success_temp.append(globals()["Success_sM{}_{}".format(n+1,m)]*100/Repeat)
    Success_sM.append(np.mean(Success_temp))
    Success_sM_list += Success_temp

In [ ]:
size_biasG = []
size_biasM = []
size_biasG_list = []
size_biasM_list = []
for n in range(5):
    Winfin_AG_temp = []
    Winfin_BG_temp = []
    Winfin_AM_temp = []
    Winfin_BM_temp = []
    for m in range(5):
        Winfin_AG_temp.append(globals()["Win_sG{}_{}".format(n+1,m)][0])
        Winfin_BG_temp.append(globals()["Win_sG{}_{}".format(n+1,m)][1])
        
        Winfin_AM_temp.append(globals()["Win_sM{}_{}".format(n+1,m)][0])
        Winfin_BM_temp.append(globals()["Win_sM{}_{}".format(n+1,m)][1])
        
    size_biasG_list += list((np.array(Winfin_AG_temp) - np.array(Winfin_BG_temp)))
    size_biasM_list += list((np.array(Winfin_AM_temp) - np.array(Winfin_BM_temp)))
    
    size_biasG.append(np.mean((np.array(Winfin_AG_temp) - np.array(Winfin_BG_temp))))
    size_biasM.append(np.mean((np.array(Winfin_AM_temp) - np.array(Winfin_BM_temp))))

In [ ]:
Success_sM_F = []
for n in range(5):
    temp = Success_sM_list[5*n:5*(n+1)]
    Success_sM_F.append(np.mean(temp))

Success_sG_F = []
for n in range(5):
    temp = Success_sG_list[5*n:5*(n+1)]
    Success_sG_F.append(np.mean(temp))   

In [ ]:
data_dic = {}
data_dic['Size bias'] = [1,2,3,4,5,1,2,3,4,5]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(5):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(5):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')

data_dic['Performance (%)'] = np.array(Success_sM_F+Success_sG_F)

df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.lineplot(data = df, x='Size bias', y='Performance (%)', marker='o', markersize=10, palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.savefig('Fig-Sizef.png', bbox_inches='tight')
plt.show()

In [ ]:
data_dic = {}
data_dic['Size bias'] = []
for n in range(10):
    m = n%5+1
    data_dic['Size bias'] += [m,m,m,m,m]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(25):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(25):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')
    
data_dic['Bias ratio (%)'] = np.array(list(size_biasM_list) +list(size_biasG_list))
df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.lineplot(data = df, x='Size bias', y='Bias ratio (%)', marker='o', markersize=10, palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(0.55,0.85))
plt.ylim([0,50])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.savefig('Fig-Size_biasf.png', bbox_inches='tight')
plt.show()

In [ ]:
data_dic = {}
data_dic['Size bias'] = [1,2,3,4,5,1,2,3,4,5]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(5):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(5):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')
    
data_dic['Bias ratio (%)'] = np.array(list(size_biasM) +list(size_biasG))
df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.lineplot(data = df, x='Size bias', y='Bias ratio (%)', marker='o', markersize=10, palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(0.55,0.85))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.savefig('Fig-Size_biasf.png', bbox_inches='tight')
plt.show()

In [ ]:
for n in range(5):
    for m in range(5):
        with open(path+path_results+'//Results-compG_v4%d-%d.pkl' % (n+1,m), 'rb') as file:
            Success = pickle.load(file)
            Fail = pickle.load(file)
            If_fail = pickle.load(file)
            Winfin = pickle.load(file)
            
            globals()["CA3_R_list_cG{}_{}".format(n+1,m)] = pickle.load(file)
            globals()["CA1_R_list_cG{}_{}".format(n+1,m)] = pickle.load(file)

In [ ]:
for n in range(5):
    for m in range(5):
        with open(path+path_results+'//Results-compM_v4%d-%d.pkl' % (n+1,m), 'rb') as file:
            Success = pickle.load(file)
            Fail = pickle.load(file)
            If_fail = pickle.load(file)
            Winfin = pickle.load(file)
            
            globals()["CA3_R_list_cM{}_{}".format(n+1,m)] = pickle.load(file)
            globals()["CA1_R_list_cM{}_{}".format(n+1,m)] = pickle.load(file)

In [ ]:
with open('Cue_and_Target_Numcomp_v4.pkl', 'rb') as file:
    Target_Data = pickle.load(file)
    Cue_Data = pickle.load(file)

In [ ]:
# competition num 2
Target_temp0 = Target_Data[:5]
# competition num 3
Target_temp1 = Target_Data[5:10]
# competition num 4
Target_temp2 = Target_Data[10:15]
# competition num 5
Target_temp3 = Target_Data[15:20]
# competition num 6
Target_temp4 = Target_Data[20:25]

In [ ]:
# Target data, engram 정리
for n in range(5):
    for m in range(5):
        globals()["Data_comp{}_{}".format(n,m)] = []
        globals()["CA3_comp{}_{}".format(n,m)] = []
        for j, comp in enumerate(globals()["Target_temp{}".format(n)][m]):
            globals()["Data_comp{}_{}".format(n,m)].append(np.where(Filt_idata[:,comp] !=0)[0])
            globals()["CA3_comp{}_{}".format(n,m)].append(Af_py[comp])

In [ ]:
N = 2
for n in range(5):
    N = N+n
    for m in range(5):
        globals()["Success_cG{}_{}".format(n+1,m)], globals()["W_cG{}_{}".format(n+1,m)], globals()["If_cG{}_{}".format(n+1,m)], globals()["F_cG{}_{}".format(n+1,m)], globals()["W_cG{}_{}".format(n+1,m)] =  calculate_success(
        globals()["CA3_R_list_cG{}_{}".format(n+1,m)], 
        globals()["CA1_R_list_cG{}_{}".format(n+1,m)], 
        globals()["CA3_comp{}_{}".format(n,m)], 
        globals()["Data_comp{}_{}".format(n,m)], N)
        
Success_cG = []
Success_cG_list = []
for n in range(5):
    Success_temp = []
    for m in range(5):
        Success_temp.append(globals()["Success_cG{}_{}".format(n+1,m)]*100/Repeat)
    Success_cG.append(np.mean(Success_temp))
    Success_cG_list += Success_temp

In [ ]:
N = 2
for n in range(5):
    N = N+n
    for m in range(5):
        globals()["Success_cM{}_{}".format(n+1,m)], globals()["W_cM{}_{}".format(n+1,m)], globals()["If_cM{}_{}".format(n+1,m)], globals()["F_cM{}_{}".format(n+1,m)], globals()["W_cM{}_{}".format(n+1,m)] =  calculate_success(
        globals()["CA3_R_list_cM{}_{}".format(n+1,m)], 
        globals()["CA1_R_list_cM{}_{}".format(n+1,m)], 
        globals()["CA3_comp{}_{}".format(n,m)], 
        globals()["Data_comp{}_{}".format(n,m)], N)
        
Success_cM = []
Success_cM_list = []
for n in range(5):
    Success_temp = []
    for m in range(5):
        Success_temp.append(globals()["Success_cM{}_{}".format(n+1,m)]*100/Repeat)
    Success_cM.append(np.mean(Success_temp))
    Success_cM_list += Success_temp

In [ ]:
Success_cM_F = []
for n in range(5):
    temp = Success_cM_list[5*n:5*(n+1)]
    Success_cM_F.append(np.mean(temp))
Success_cG_F = []
for n in range(5):
    temp = Success_cG_list[5*n:5*(n+1)]
    Success_cG_F.append(np.mean(temp))   

In [ ]:
data_dic = {}
data_dic['Number of competitive engram'] = [2,3,4,5,6]+[2,3,4,5,6]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(5):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(5):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')

data_dic['Performance (%)'] = np.array(Success_cM_F+Success_cG_F)

df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.lineplot(data = df, x='Number of competitive engram', y='Performance (%)', marker='o', markersize=10, palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.savefig('Fig-Compf.png', bbox_inches='tight')
plt.show()

In [ ]:
data_dic = {}
data_dic['Number of competitive engram'] = []
for n in range(10):
    m = n%5
    data_dic['Number of competitive engram'] += [m+2,m+2,m+2,m+2,m+2]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(25):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(25):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')

data_dic['Performance (%)'] = Success_cM_list+Success_cG_list

df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.lineplot(data = df, x='Number of competitive engram', y='Performance (%)', marker='o', markersize=10, palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.show()

In [ ]:
data_dic = {}
data_dic['Number of competitive engram'] = []
for n in range(10):
    m = n%5
    data_dic['Number of competitive engram'] += [m+2,m+2,m+2,m+2,m+2]
data_dic['Mutual exclusive / Global inhibition'] = []
for n in range(25):
    data_dic['Mutual exclusive / Global inhibition'].append('Mutual exclusive')
for n in range(25):
    data_dic['Mutual exclusive / Global inhibition'].append('Global inhibition')

data_dic['Performance (%)'] = Success_cM_list+Success_cG_list

df = pd.DataFrame(data_dic)

fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.scatterplot(data = df, x='Number of competitive engram', y='Performance (%)', marker='o', palette={'Mutual exclusive':'cornflowerblue','Global inhibition':'salmon'}, hue=df['Mutual exclusive / Global inhibition'], linewidth=2.5)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))
plt.ylim([0,100])
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
#plt.savefig('Fig-Comp.png', bbox_inches='tight')
plt.show()

In [ ]:
with open('Results_various_cond3.pkl' ,'wb') as file:
    pickle.dump(Success_bM_list, file)
    pickle.dump(Success_bG_list, file)
    
    pickle.dump(Success_oM_list, file)
    pickle.dump(Success_oG_list, file)
    
    pickle.dump(Success_sM_list, file)
    pickle.dump(Success_sG_list, file)
    
    pickle.dump(size_biasM_list, file)
    pickle.dump(size_biasG_list, file)
    
    pickle.dump(Success_cM_list, file)
    pickle.dump(Success_cG_list, file)
